In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from io import StringIO,BytesIO

In [6]:
def extract_flight_info(raw_url, file_type='csv'):
    print("Fetching data from:", raw_url)

    # Fetch the raw file
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(raw_url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch file. HTTP Status Code: {response.status_code}")

    # Load the file into a DataFrame
    if file_type == 'csv':
        dataframe = pd.read_csv(StringIO(response.text))
    elif file_type in ('xls', 'xlsx'):
        from io import BytesIO
        dataframe = pd.read_excel(BytesIO(response.content))
    else:
        raise ValueError("Unsupported file type. Only 'csv' and 'excel' are supported.")

    return dataframe


# Correct raw URL
raw_url = "https://raw.githubusercontent.com/Qusid/MphasisQuantumChallenge/main/Data/PRMI_DM_ALL_PNRs.csv"


flightPNRs = extract_flight_info(raw_url)

# Call the function and display results
print(flightPNRs)  # Preview the first few rows


Fetching data from: https://raw.githubusercontent.com/Qusid/MphasisQuantumChallenge/main/Data/PRMI_DM_ALL_PNRs.csv
       RECLOC CREATION_DTZ CABIN_CD  COS_CD OPER_OD_ORIG_CD OPER_OD_DEST_CD  \
0           0   2027-06-11        Y    23.0             EBG             TPH   
1           1   2027-09-03        Y    21.0             KHL             RHP   
2           1   2027-09-03        Y    21.0             KHL             RHP   
3           1   2027-09-03        Y    20.0             RHP             KHL   
4           1   2027-09-03        Y    20.0             RHP             KHL   
...       ...          ...      ...     ...             ...             ...   
43827   14185   2027-02-25        Y    10.0             PTP             NMK   
43828   14186   2027-10-09        Y    14.0             ALN             NAD   
43829   14186   2027-10-09        Y    14.0             ALN             NAD   
43830   14186   2027-10-09        Y    14.0             NAD             ALN   
43831   14186   

In [31]:
#Create CVM function 

##Ideally I want to make it so you can adjust the weights manually
def CVM(df):
    '''
    
    :param dataframe: The data frame containing flights 
    :return: Dataframe with attached CVM scores 
    '''
    # Define weights for the CVM score calculation
    weights = {
        'Cabin': 0.3,  # Weight for cabin code
        'Group_Size': 0.1,  # Weight for group size
        'Connection': 0.1,  # Weight for connection time impact
        'Delay': 0.1,  # Weight for delay penalty
        'Seat_Availability': 0.05  # Weight for seat availability (if applicable)
    }

    # Ensure the DataFrame contains the expected columns
    # Here are the mappings based on the dataseet:
    df['Cabin_Score'] = df['CABIN_CD'].map({ 'C': 0.7, 'Y': 0.5})  # Scores to be adjusted C is premium Y is economy
    df['Group_Size_Impact'] = 1 / (1 + df['PAX_CNT'])  # Smaller group sizes get higher scores
    # Ensure datetime conversion for delay calculation
    df['ARR_DTML'] = pd.to_datetime(df['ARR_DTML'], errors='coerce')
    df['DEP_DTML'] = pd.to_datetime(df['DEP_DTML'], errors='coerce')
    df['CONN_TIME_MINS'] = df['CONN_TIME_MINS'].fillna(0) # Fill NaN values with 0
    df['Connection_Impact'] = df['CONN_TIME_MINS'] / 60  # Normalize connection time (assuming 60 minutes is ideal)
    df['Delay_Penalty'] = (df['ARR_DTML'] - df['DEP_DTML']).dt.total_seconds() / 3600  # Calculate delay in hours
    df['Delay_Penalty'] = df['Delay_Penalty'] / df['Delay_Penalty'].max()  # Normalize delay
    df['Available_Seat_Impact'] = 1  # Set a default value if availability is not in your dataset

    # Calculate CVM Score using the weighted formula
    df['CVM_Score'] = (
        weights['Cabin'] * df['Cabin_Score'] +
        weights['Group_Size'] * df['Group_Size_Impact'] +
        weights['Connection'] * df['Connection_Impact'] +
        weights['Delay'] * (1 - df['Delay_Penalty']) +  # Higher delay reduces score
        weights['Seat_Availability'] * df['Available_Seat_Impact']
    )
    
    return df

In [32]:
## Make a smaller copy of the data to test CVM 
df = flightPNRs.copy()
df = CVM(df)
print(df)


       RECLOC CREATION_DTZ CABIN_CD  COS_CD OPER_OD_ORIG_CD OPER_OD_DEST_CD  \
0           0   2027-06-11        Y    23.0             EBG             TPH   
1           1   2027-09-03        Y    21.0             KHL             RHP   
2           1   2027-09-03        Y    21.0             KHL             RHP   
3           1   2027-09-03        Y    20.0             RHP             KHL   
4           1   2027-09-03        Y    20.0             RHP             KHL   
...       ...          ...      ...     ...             ...             ...   
43827   14185   2027-02-25        Y    10.0             PTP             NMK   
43828   14186   2027-10-09        Y    14.0             ALN             NAD   
43829   14186   2027-10-09        Y    14.0             ALN             NAD   
43830   14186   2027-10-09        Y    14.0             NAD             ALN   
43831   14186   2027-10-09        Y    14.0             NAD             ALN   

                     DEP_KEY      DEP_DT ORIG_CD DE